In [2]:
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import numpy as np
from vit_model import SingleImageTransformer
from dataset import BarLinkageDataset 
import matplotlib.pyplot as plt
torch.set_float32_matmul_precision('medium')

from dataset_generation.curve_plot import get_pca_inclination, rotate_curve
import scipy.spatial.distance as sciDist
from tqdm import tqdm
import requests
import time
import matplotlib.pyplot as plt
import os
import json
import torch.nn.functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# Headless simulator version
index = 0 # local server index 
API_ENDPOINT = f"http://localhost:4000/simulation"
HEADERS = {"Content-Type": "application/json"}
speedscale = 1
steps = 360
minsteps = int(steps*20/360)

In [4]:
checkpoint_path = "weights/transformer_weights_VIT/d1024_h32_n6_bs512_lr5e-05_best.pth"
data_dir = "/home/anurizada/Documents/processed_dataset"
batch_size = 1

dataset = BarLinkageDataset(data_dir=data_dir)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [5]:
checkpoint = torch.load(checkpoint_path, map_location=device)
model_config = checkpoint['model_config']

# Initialize model
model = SingleImageTransformer(
    tgt_seq_len=model_config['tgt_seq_len'],
    d_model=model_config['d_model'],
    h=model_config['h'],
    N=model_config['N'],
    num_labels=model_config['num_labels'],
    vocab_size=model_config['vocab_size'],
).to(device)

# Load weights
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

/home/anurizada/anaconda3/envs/transformer/lib/python3.10/site-packages/torch/nn/modules/transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


SingleImageTransformer(
  (tgt_embed): InputEmbeddings(
    (embedding): Embedding(221, 1024)
  )
  (decoder_positional_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (image_encoder): ViTEncoder(
    (patch_embed): Conv2d(1, 1024, kernel_size=(8, 8), stride=(8, 8), bias=False)
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
          )
          (linear1): Linear(in_features=1024, out_features=4096, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=4096, out_features=1024, bias=True)
          (norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout

In [7]:
# ===============================
# CONFIG
# ===============================
MAX_SAMPLES = 50
TEMPERATURES = [0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 1.0, 1.3, 1.5, 1.7, 2.0]

SOS_TOKEN, EOS_TOKEN, PAD_TOKEN = 0, 1, 2
NUM_SPECIAL_TOKENS = 3
NUM_MECH_TYPES = 17
BIN_OFFSET = NUM_SPECIAL_TOKENS + NUM_MECH_TYPES  # = 20

# Define externally:
# API_ENDPOINT, HEADERS, speedscale, steps, minsteps

label_mapping_path = "/home/anurizada/Documents/processed_dataset/label_mapping.json"
coupler_mapping_path = "/home/anurizada/Documents/transformer/BSIdict.json"

with open(label_mapping_path, "r") as f:
    label_mapping = json.load(f)
index_to_label = label_mapping["index_to_label"]

with open(coupler_mapping_path, "r") as f:
    coupler_mapping = json.load(f)

def coupler_index_for(mech_type: str) -> int:
    """Return coupler curve index from BSIdict.json."""
    if mech_type in coupler_mapping and "c" in coupler_mapping[mech_type]:
        cvec = coupler_mapping[mech_type]["c"]
        if isinstance(cvec, list) and 1 in cvec:
            return cvec.index(1)
    return -1

# ===============================
# BINNER
# ===============================
class CoordinateBinner:
    def __init__(self, kappa=1.0, num_bins=200):
        self.kappa = kappa
        self.num_bins = num_bins
        self.bin_edges = np.linspace(-kappa, kappa, num_bins + 1)
        self.bin_centers = (self.bin_edges[:-1] + self.bin_edges[1:]) / 2

    def bin_to_value_torch(self, bin_index_tensor):
        bin_index_tensor = torch.clamp(bin_index_tensor, 0, self.num_bins - 1)
        device = bin_index_tensor.device
        bin_centers_tensor = torch.tensor(self.bin_centers, device=device, dtype=torch.float32)
        return bin_centers_tensor[bin_index_tensor]

binner = CoordinateBinner(kappa=1.0, num_bins=201)

# ===============================
# AUTOREGRESSIVE PREDICTION
# ===============================
def predict_autoregressive(model, image, max_seq_len, device, temperature=1.0, top_k=None):
    model.eval()
    with torch.no_grad():
        decoder_input = torch.tensor([[SOS_TOKEN]], device=device, dtype=torch.long)
        for _ in range(max_seq_len):
            T = decoder_input.size(1)
            causal_mask = torch.tril(torch.ones(T, T, device=device)).bool()

            logits = model(decoder_input, causal_mask, image, None)
            next_logits = logits[:, -1, :] / float(temperature)
            probs = F.softmax(next_logits, dim=-1)

            if top_k is None:
                next_token = torch.argmax(probs, dim=-1)
            else:
                k = min(int(top_k), probs.size(-1))
                topk_probs, topk_idx = torch.topk(probs, k=k, dim=-1)
                sampled = torch.multinomial(topk_probs, num_samples=1)
                next_token = topk_idx.gather(-1, sampled).squeeze(1)

            token = int(next_token.item())
            decoder_input = torch.cat([decoder_input, next_token.unsqueeze(1)], dim=1)
            if token == EOS_TOKEN:
                break
    return decoder_input.squeeze(0).cpu().numpy()

# ===============================
# MAIN LOOP
# ===============================
print("Starting multi-temperature coupler curve generation...")
os.makedirs("results_coupler", exist_ok=True)

for i, batch in enumerate(tqdm(dataloader, total=MAX_SAMPLES, desc="Simulating")):
    if i >= MAX_SAMPLES:
        break

    image = batch["images"].to(device)
    gt_tokens = batch["labels_discrete"][0].numpy()

    # --- Ground Truth ---
    gt_mech_idx = gt_tokens[0] - NUM_SPECIAL_TOKENS
    gt_mech_name = index_to_label.get(str(gt_mech_idx), "UNKNOWN")
    gt_coord_tokens = [t for t in gt_tokens[1:] if t >= BIN_OFFSET]
    if len(gt_coord_tokens) < 4:
        continue
    gt_coords_tensor = torch.tensor(gt_coord_tokens) - BIN_OFFSET
    gt_coords_float = binner.bin_to_value_torch(gt_coords_tensor).cpu().numpy()
    if gt_coords_float.size % 2 == 1:
        gt_coords_float = gt_coords_float[:-1]
    gt_points = gt_coords_float.reshape(-1, 2)

    # Create subfolder for this sample
    sample_dir = f"results_coupler/sample_{i:03d}_{gt_mech_name}"
    os.makedirs(sample_dir, exist_ok=True)

    # --- Simulate GT coupler ---
    ex_gt = {
        "params": gt_points.tolist(),
        "type": gt_mech_name,
        "speedScale": speedscale,
        "steps": steps,
        "relativeTolerance": 0.1,
    }
    try:
        temp = requests.post(url=API_ENDPOINT, headers=HEADERS, data=json.dumps([ex_gt])).json()
        P = np.array(temp[0]["poses"]) if isinstance(temp, list) and temp and "poses" in temp[0] else None
    except Exception as e:
        print(f"GT sim failed: {e}")
        continue
    if P is None or P.shape[0] < minsteps:
        continue

    coup_idx_gt = coupler_index_for(gt_mech_name)
    if coup_idx_gt < 0:
        continue
    original_x, original_y = P[:, coup_idx_gt, 0], P[:, coup_idx_gt, 1]
    orig_phi = -get_pca_inclination(original_x, original_y)
    orig_denom = np.sqrt(np.var(original_x) + np.var(original_y)) + 1e-8
    ox_mean, oy_mean = np.mean(original_x), np.mean(original_y)

    print(f"\n=== Sample {i} | GT={gt_mech_name} ===")
    print("GT joint positions:")
    for j, (x, y) in enumerate(gt_points):
        print(f"  J{j}: ({x:.3f}, {y:.3f})")

    # --- Multiple Predictions (different temperatures) ---
    for temp_val in TEMPERATURES:
        pred_tokens = predict_autoregressive(model, image, model_config["tgt_seq_len"], device, temperature=temp_val, top_k=3)
        mech_idx = pred_tokens[1] - NUM_SPECIAL_TOKENS if len(pred_tokens) > 1 else -1
        mech_name = index_to_label.get(str(mech_idx), "UNKNOWN")

        coord_tokens = [t for t in pred_tokens if t >= BIN_OFFSET]
        if len(coord_tokens) < 4:
            continue
        coords_tensor = torch.tensor(coord_tokens) - BIN_OFFSET
        coords_float = binner.bin_to_value_torch(coords_tensor).cpu().numpy()
        if coords_float.size % 2 == 1:
            coords_float = coords_float[:-1]
        pred_points = coords_float.reshape(-1, 2)

        print(f"\n[Temp={temp_val:.1f}] Pred mech: {mech_name}")
        print("Pred joint positions:")
        for j, (x, y) in enumerate(pred_points):
            print(f"  J{j}: ({x:.3f}, {y:.3f})")

        # --- Simulate predicted coupler ---
        ex_pred = {
            "params": pred_points.tolist(),
            "type": mech_name,
            "speedScale": speedscale,
            "steps": steps,
            "relativeTolerance": 0.1,
        }
        try:
            temp = requests.post(url=API_ENDPOINT, headers=HEADERS, data=json.dumps([ex_pred])).json()
            Pp = np.array(temp[0]["poses"]) if isinstance(temp, list) and temp and "poses" in temp[0] else None
        except Exception as e:
            print(f"Pred sim failed (temp={temp_val}): {e}")
            continue
        print(Pp)
        
        if Pp is None:
            continue

        if P.shape[0] < minsteps:
            continue

        coup_idx_pred = coupler_index_for(mech_name)
        if coup_idx_pred < 0:
            continue
        generated_x, generated_y = Pp[:, coup_idx_pred, 0], Pp[:, coup_idx_pred, 1]
        if np.isnan(generated_x).any() or np.isinf(generated_x).any():
            continue

        # --- Align predicted to GT ---
        gen_phi = -get_pca_inclination(generated_x, generated_y)
        rotation = gen_phi - orig_phi
        generated_x, generated_y = rotate_curve(generated_x, generated_y, rotation)
        gen_denom = np.sqrt(np.var(generated_x) + np.var(generated_y)) + 1e-8
        scale = orig_denom / gen_denom
        generated_x *= scale
        generated_y *= scale
        gx_mean, gy_mean = np.mean(generated_x), np.mean(generated_y)
        generated_x -= (gx_mean - ox_mean)
        generated_y -= (gy_mean - oy_mean)

        # --- Plot ---
        plt.figure(figsize=(6, 6))
        plt.plot(original_x, original_y, "r-", label=f"GT Coupler ({gt_mech_name})")
        plt.plot(generated_x, generated_y, "g--", label=f"Pred Coupler ({mech_name})")

        # Plot joint points
        plt.scatter(gt_points[:, 0], gt_points[:, 1], color="red", s=40, zorder=5)
        for j, (x, y) in enumerate(gt_points):
            plt.text(x, y, f"J{j}", color="red", fontsize=9, weight="bold")

        plt.scatter(pred_points[:, 0], pred_points[:, 1], color="green", s=40, zorder=5)
        for j, (x, y) in enumerate(pred_points):
            plt.text(x, y, f"J{j}", color="green", fontsize=9, weight="bold")

        plt.axis("equal")
        plt.legend()
        plt.title(f"Sample {i} | GT={gt_mech_name} | Pred={mech_name} | Temp={temp_val}")
        plt.tight_layout()
        save_path = os.path.join(sample_dir, f"temp_{temp_val:.1f}.png")
        plt.savefig(save_path)
        plt.close()
        print(f"✅ Saved: {save_path}")

print("✅ Finished all temperature variations.")


Starting multi-temperature coupler curve generation...


Simulating:   0%|          | 0/50 [00:00<?, ?it/s]


=== Sample 0 | GT=RRRR ===
GT joint positions:
  J0: (0.478, 0.816)
  J1: (-0.716, -0.219)
  J2: (0.488, 0.070)
  J3: (0.637, 0.199)
  J4: (0.239, -0.557)

[Temp=0.1] Pred mech: RRRR
Pred joint positions:
  J0: (0.478, 0.816)
  J1: (-0.716, -0.219)
  J2: (0.488, 0.070)
  J3: (0.637, 0.199)
  J4: (0.239, -0.557)
[[[ 0.47761193  0.81592041]
  [-1.06840365  0.48971845]
  [ 0.48756218  0.06965174]
  [ 0.30575519  0.14682139]
  [-0.42622189 -0.29418706]]

 [[ 0.47761193  0.81592041]
  [-1.06247517  0.46278644]
  [ 0.48756218  0.06965174]
  [ 0.32594549  0.18318138]
  [-0.38506677 -0.29088663]]

 [[ 0.47761193  0.81592041]
  [-1.05607757  0.43596199]
  [ 0.48756218  0.06965174]
  [ 0.34071373  0.20172924]
  [-0.35447981 -0.29524647]]

 [[ 0.47761193  0.81592041]
  [-1.04921279  0.40925329]
  [ 0.48756218  0.06965174]
  [ 0.35367702  0.21485401]
  [-0.32709617 -0.30169877]]

 [[ 0.47761193  0.81592041]
  [-1.04188293  0.38266846]
  [ 0.48756218  0.06965174]
  [ 0.36560953  0.22501095]
  [-0.

Simulating:   2%|▏         | 1/50 [00:02<02:21,  2.89s/it]


[Temp=2.0] Pred mech: RRRR
Pred joint positions:
  J0: (0.478, -0.716)
  J1: (-0.219, -0.488)
  J2: (0.816, -0.557)
  J3: (0.587, -0.557)
  J4: (0.199, -0.408)
[[[ 4.77611929e-01 -7.16417909e-01]
  [ 6.44891301e-01 -2.60486874e-03]
  [ 8.15920413e-01 -5.57213902e-01]
  [ 8.70888998e-01 -7.79370152e-01]
  [ 8.69989793e-01 -3.63598349e-01]]

 [[ 4.77611929e-01 -7.16417909e-01]
  [ 6.32408068e-01  2.05841660e-04]
  [ 8.15920413e-01 -5.57213902e-01]
  [ 8.23171899e-01 -7.85954718e-01]
  [ 8.41010014e-01 -3.70564778e-01]]

 [[ 4.77611929e-01 -7.16417909e-01]
  [ 6.19877683e-01  2.79826153e-03]
  [ 8.15920413e-01 -5.57213902e-01]
  [ 8.01192275e-01 -7.85595221e-01]
  [ 8.24014615e-01 -3.70449293e-01]]

 [[ 4.77611929e-01 -7.16417909e-01]
  [ 6.07303963e-01  5.17160118e-03]
  [ 8.15920413e-01 -5.57213902e-01]
  [ 7.85205266e-01 -7.83999097e-01]
  [ 8.09823840e-01 -3.68955814e-01]]

 [[ 4.77611929e-01 -7.16417909e-01]
  [ 5.94690736e-01  7.32513768e-03]
  [ 8.15920413e-01 -5.57213902e-01]
  [

...


[Temp=2.0] Pred mech: RRRR
Pred joint positions:
  J0: (0.119, 0.448)
  J1: (0.139, -0.776)
  J2: (0.338, -0.299)
  J3: (-0.995, -0.308)
  J4: (0.498, -0.408)
[[[ 0.11940298  0.44776121]
  [ 0.13930348 -0.77611941]
  [ 0.33830845 -0.29850745]
  [-0.99502486 -0.3084577 ]
  [ 0.49751243 -0.40796021]]

 [[ 0.11940298  0.44776121]
  [ 0.16066011 -0.7755857 ]
  [ 0.33830845 -0.29850745]
  [-0.99365705 -0.35970091]
  [ 0.50185308 -0.39160365]]

 [[ 0.11940298  0.44776121]
  [ 0.18200417 -0.77467934]
  [ 0.33830845 -0.29850745]
  [-0.9902005  -0.41226459]
  [ 0.50519622 -0.37542785]]

 ...

 [[ 0.11940298  0.44776121]
  [ 0.07522324 -0.77548364]
  [ 0.33830845 -0.29850745]
  [-0.98820992 -0.16350475]
  [ 0.47849944 -0.45732776]]

 [[ 0.11940298  0.44776121]
  [ 0.09657854 -0.77606837]
  [ 0.33830845 -0.29850745]
  [-0.9921409  -0.21029613]
  [ 0.48582782 -0.440897  ]]

 [[ 0.11940298  0.44776121]
  [ 0.11794078 -0.77628032]
  [ 0.33830845 -0.29850745]
  [-0.99446559 -0.25863148]
  [ 0.492169

Simulating:  98%|█████████▊| 49/50 [02:33<00:02,  2.92s/it]


[Temp=2.0] Pred mech: RRRR
Pred joint positions:
  J0: (0.169, -0.159)
  J1: (0.478, 0.259)
  J2: (-0.498, 0.259)
  J3: (-0.189, 0.687)
  J4: (0.706, 0.060)
[[[ 0.16915423 -0.15920398]
  [ 0.47761193  0.25870648]
  [-0.49751243  0.25870648]
  [-0.18905473  0.68656719]
  [ 0.70646769  0.05970149]]

 [[ 0.16915423 -0.15920398]
  [ 0.47027141  0.26402616]
  [-0.49751243  0.25870648]
  [-0.19644948  0.69180236]
  [ 0.69915239  0.06505018]]

 [[ 0.16915423 -0.15920398]
  [ 0.46283916  0.26921692]
  [-0.49751243  0.25870648]
  [-0.20393457  0.69691075]
  [ 0.69174472  0.07026922]]

 ...

 [[ 0.16915423 -0.15920398]
  [ 0.49906094  0.24199032]
  [-0.49751243  0.25870648]
  [-0.16743434  0.67011794]
  [ 0.727837    0.04289371]]

 [[ 0.16915423 -0.15920398]
  [ 0.49200889  0.24768688]
  [-0.49751243  0.25870648]
  [-0.174545    0.67572324]
  [ 0.7208122   0.0486216 ]]

 [[ 0.16915423 -0.15920398]
  [ 0.48485849  0.2532595 ]
  [-0.49751243  0.25870648]
  [-0.18175251  0.68120691]
  [ 0.71368837

Simulating: 100%|██████████| 50/50 [02:36<00:00,  3.12s/it]


[Temp=2.0] Pred mech: RRRR
Pred joint positions:
  J0: (-0.846, 0.478)
  J1: (0.746, 0.279)
  J2: (-0.985, 0.667)
  J3: (-0.965, 0.866)
  J4: (0.687, 0.448)
[[[-0.84577113  0.47761193]
  [-2.17785781 -0.41666917]
  [-0.98507464  0.66666669]
  [-0.81865408  0.77758647]
  [-2.00138067 -0.44881329]]

 [[-0.84577113  0.47761193]
  [-2.16204757 -0.43978108]
  [-0.98507464  0.66666669]
  [-0.80890036  0.76133254]
  [-1.98541014 -0.47103238]]

 [[-0.84577113  0.47761193]
  [-2.14583638 -0.46261355]
  [-0.98507464  0.66666669]
  [-0.80355704  0.75063312]
  [-1.96892477 -0.4922734 ]]

 ...

 [[-0.84577113  0.47761193]
  [ 0.43768691  1.44038348]
  [-0.98507464  0.66666669]
  [-1.18256778  0.63511579]
  [ 0.27537764  1.51675839]]

 [[-0.84577113  0.47761193]
  [ 0.42068875  1.46263627]
  [-0.98507464  0.66666669]
  [-1.17874964  0.61677686]
  [ 0.25650561  1.53489499]]

 [[-0.84577113  0.47761193]
  [ 0.40330482  1.48458902]
  [-0.98507464  0.66666669]
  [-1.17080024  0.592471  ]
  [ 0.23708535